[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](
https://colab.research.google.com/github/Abhishek-Gupta-GitHub//confocal_microscopy-copilot/blob/main/main_demo.ipynb
)


imports

In [18]:
from pathlib import Path

import numpy as np

from copilot.config import DEFAULT_META
from copilot.io_utils import (
    load_stack,
    save_stack,
    load_metadata,
    save_metadata,
    save_json,
)
from copilot.digital_twin import simulate_confocal_stack
from copilot.orchestrator import Orchestrator
from copilot.detection_tracking import DetectionTrackingWorker
from copilot.physics_analysis import PhysicsAnalyst
from copilot.chat_explainer import ChatExplainer


This cell imports the config, IO helpers, digital twin, and all four agents. The notebook will show how they interact end‑to‑end on one dataset.

In [19]:
data_dir = Path("data")
stack_path = data_dir / "example_stack.tif"
meta_path = data_dir / "example_stack_meta.json"

data_dir.mkdir(exist_ok=True)

if stack_path.exists() and meta_path.exists():
    print("Loading existing example stack...")
    stack_3d = load_stack(stack_path)
    metadata = load_metadata(meta_path)
else:
    print("Simulating example stack with digital twin...")
    metadata = DEFAULT_META.copy()
    stack_3d, _ = simulate_confocal_stack(metadata, twin_settings={
        "n_particles": 200,
        "D": 0.2,
        "n_frames": 100,
        "dt": metadata["frame_interval_s"],
    })
    save_stack(stack_path, stack_3d)
    save_metadata(meta_path, metadata)

stack_3d.shape, metadata


Loading existing example stack...


((100, 32, 64, 64),
 {'voxel_size_um': [0.1, 0.1, 0.2],
  'img_shape_xyz': [32, 64, 64],
  'psf_sigma_xyz_vox': [2.0, 1.0, 1.0],
  'frame_interval_s': 0.1,
  'noise_std': 5.0,
  'z_att_um': 50.0,
  'bleach_tau_s': 80.0,
  'box_size_um': [30.0, 30.0, 30.0]})

This step either loads a real confocal dataset from data/ or simulates a synthetic example using the digital twin. The result is a 4D array (T, Z, Y, X) plus a metadata dict.

In [20]:
def compute_quick_stats(stack_3d):
    mean_signal = float(stack_3d.mean())
    std_signal = float(stack_3d.std())
    snr_est = mean_signal / (std_signal + 1e-6)
    # crude placeholders for now
    density_est = 200
    D_est = 0.2
    search_range_um = 1.0
    return {
        "snr_est": snr_est,
        "density_est": density_est,
        "D_est": D_est,
        "search_range_um": search_range_um,
    }

user_query = "Analyze diffusion and comment on depth and bleaching."

orch = Orchestrator()
quick_stats = compute_quick_stats(stack_3d)
metadata["n_frames"] = stack_3d.shape[0]

plan = orch.make_plan(user_query, metadata, quick_stats)
plan.to_dict()


{'pipeline_type': 'diffusion',
 'twin_needed': True,
 'twin_settings': {'n_particles': 200, 'D': 0.2, 'n_frames': 100, 'dt': 0.1},
 'detection_params_initial': {'min_sigma': 1,
  'max_sigma': 4,
  'threshold': -2.6271502961855232e-05,
  'minmass': 200.0},
 'tracking_params_initial': {'search_range': 1.0, 'memory': 2}}

Here the orchestrator (Agent 1) takes the user question, metadata, and quick statistics (SNR, density, D guess) and returns a Plan. The plan contains which pipeline to run, whether to use the digital twin, and initial detection/tracking parameters.

In [21]:
import importlib
from copilot import detection_tracking
importlib.reload(detection_tracking)
from copilot.detection_tracking import DetectionTrackingWorker


In [30]:
det_worker = DetectionTrackingWorker()
dt_result = det_worker.run(stack_3d, plan)

trajectories = dt_result["trajectories"]
quality_metrics = dt_result["quality_metrics"]

# Show only a small preview instead of the full DataFrame
print("Number of tracks:", quality_metrics["n_tracks"])
print("Detections per frame (first 5):",
      dict(list(quality_metrics["detections_per_frame"].items())[:5]))

trajectories.head()  # this will render only the first 5 rows







C:\Users\akumarg\AppData\Roaming\Python\Python311\site-packages\trackpy\feature.py:425: UserWarning: No maxima survived mass- and size-based filtering. Be advised that the mass computation was changed from version 0.2.4 to 0.3.0 and from 0.3.3 to 0.4.0. See the documentation and the convenience functions 'minmass_v03_change' and 'minmass_v04_change'.
  warnings.warn("No maxima survived mass- and size-based filtering. "
C:\Users\akumarg\AppData\Roaming\Python\Python311\site-packages\trackpy\feature.py:425: UserWarning: No maxima survived mass- and size-based filtering. Be advised that the mass computation was changed from version 0.2.4 to 0.3.0 and from 0.3.3 to 0.4.0. See the documentation and the convenience functions 'minmass_v03_change' and 'minmass_v04_change'.
  warnings.warn("No maxima survived mass- and size-based filtering. "
C:\Users\akumarg\AppData\Roaming\Python\Python311\site-packages\trackpy\feature.py:425: UserWarning: No maxima survived mass- and size-based filtering. Be

Frames with detections: 0
No features detected in any frame; returning empty trajectories.
Number of tracks: 0
Detections per frame (first 5): {}


C:\Users\akumarg\AppData\Roaming\Python\Python311\site-packages\trackpy\feature.py:425: UserWarning: No maxima survived mass- and size-based filtering. Be advised that the mass computation was changed from version 0.2.4 to 0.3.0 and from 0.3.3 to 0.4.0. See the documentation and the convenience functions 'minmass_v03_change' and 'minmass_v04_change'.
  warnings.warn("No maxima survived mass- and size-based filtering. "
C:\Users\akumarg\AppData\Roaming\Python\Python311\site-packages\trackpy\feature.py:425: UserWarning: No maxima survived mass- and size-based filtering. Be advised that the mass computation was changed from version 0.2.4 to 0.3.0 and from 0.3.3 to 0.4.0. See the documentation and the convenience functions 'minmass_v03_change' and 'minmass_v04_change'.
  warnings.warn("No maxima survived mass- and size-based filtering. "
C:\Users\akumarg\AppData\Roaming\Python\Python311\site-packages\trackpy\feature.py:425: UserWarning: No maxima survived mass- and size-based filtering. Be

,x,y,frame,particle


The detection/tracking worker (Agent 2) runs Trackpy (and later DeepTrack) with parameters from the plan. It returns a trajectories DataFrame and simple quality metrics such as number of tracks and track length histogram.​



In [23]:
analyst = PhysicsAnalyst()
summary = analyst.summarize(trajectories, stack_3d, metadata)

save_json(data_dir / "analysis_summary.json", summary)
summary["alpha"], summary["D"], list(summary["diagnostics"].keys())


(nan,
 nan,
 ['depth_profile_mean_intensity', 'bleaching_mean_intensity', 'crowding'])

The physics analyst (Agent 3) computes MSD, the anomalous exponent 
α
α, diffusion coefficient 
D
D, and diagnostics (depth profile, bleaching curve, crowding metric). The result is a JSON‑serialisable summary that can be logged or given to an LLM explainer.

In [25]:
explainer = ChatExplainer(llm_client=None)  # plug in a real client later
explanation = explainer.explain(user_query, summary)
print(explanation)

MSD and the fitted alpha parameter indicate the overall diffusive behaviour under your current imaging conditions. Depth and bleaching diagnostics highlight where intensity and track quality degrade.

Next experiments: (1) Restrict analysis to the depth range with stable intensity, (2) lower laser power or shorten acquisition to reduce bleaching, (3) adjust particle density or magnification to mitigate crowding and improve tracking.


The explainer (Agent 4) reads the user question plus the JSON summary and generates a short interpretation and next‑experiment suggestions. Right now it uses a dummy text; later it can call a real LLM API with the constructed prompt.